In [1]:
# Connecting personal Google Drive and giving access to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# importing modules
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


In [3]:
# loading training and testing data

# Path to the CSV file in Google Drive(temporary storage for data)
xtrain_path = '/content/drive/MyDrive/ACScode/train/X_train.csv'
ytrain_path = '/content/drive/MyDrive/ACScode/train/y_train.csv'
xtest_path = '/content/drive/MyDrive/ACScode/test/X_test.csv'
ytest_path = '/content/drive/MyDrive/ACScode/test/y_test.csv'

# Load the CSV file into a Pandas DataFrame
X_train = pd.read_csv(xtrain_path)
y_train = pd.read_csv(ytrain_path)
X_test = pd.read_csv(xtest_path)
y_test = pd.read_csv(ytest_path)

In [4]:
# convert data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).squeeze()

In [5]:
# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [6]:
# defining the architecture of the ANN model

class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()

        # Input layer
        self.layer1 = nn.Linear(input_dim, 64)

        # Hidden layer 1
        self.layer2 = nn.Linear(64, 32)

        # Hidden layer 2
        self.layer3 = nn.Linear(32, 16)

        # Output layer
        self.output = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.sigmoid(self.output(x)) # Sigmoid for binary classification
        return x

In [7]:
# Initialize model, loss function, and optimizer
input_dim = X_train.shape[1]
# model = ANNModel(input_dim).cuda()  # Move the model to the GPU
model = ANNModel(input_dim)
criterion = nn.BCELoss()  # Binary cross-entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0005) # learning rate can be modified

In [22]:
# Training loop

# setting number of epochs to 100(can be changed)
num_epochs = 100

# iterate through the epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/100], Loss: 0.4085
Epoch [2/100], Loss: 0.4077
Epoch [3/100], Loss: 0.4063
Epoch [4/100], Loss: 0.4077
Epoch [5/100], Loss: 0.4064
Epoch [6/100], Loss: 0.4067
Epoch [7/100], Loss: 0.4060
Epoch [8/100], Loss: 0.4066
Epoch [9/100], Loss: 0.4083
Epoch [10/100], Loss: 0.4061
Epoch [11/100], Loss: 0.4065
Epoch [12/100], Loss: 0.4063
Epoch [13/100], Loss: 0.4066
Epoch [14/100], Loss: 0.4073
Epoch [15/100], Loss: 0.4060
Epoch [16/100], Loss: 0.4060
Epoch [17/100], Loss: 0.4061
Epoch [18/100], Loss: 0.4056
Epoch [19/100], Loss: 0.4058
Epoch [20/100], Loss: 0.4068
Epoch [21/100], Loss: 0.4069
Epoch [22/100], Loss: 0.4058
Epoch [23/100], Loss: 0.4061
Epoch [24/100], Loss: 0.4053
Epoch [25/100], Loss: 0.4047
Epoch [26/100], Loss: 0.4054
Epoch [27/100], Loss: 0.4049
Epoch [28/100], Loss: 0.4050
Epoch [29/100], Loss: 0.4051
Epoch [30/100], Loss: 0.4048
Epoch [31/100], Loss: 0.4060
Epoch [32/100], Loss: 0.4056
Epoch [33/100], Loss: 0.4040
Epoch [34/100], Loss: 0.4061
Epoch [35/100], Loss: 0

In [23]:
# Evaluate on test data

from sklearn.metrics import accuracy_score

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).squeeze()
    y_pred_class = (y_pred > 0.5).float()
    accuracy = accuracy_score(y_test_tensor, y_pred_class)
    print(f'Test Accuracy: {accuracy:.2f}')

Test Accuracy: 0.80


In [24]:
# print the predictions

# Output results
# print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Accuracy: {accuracy}')
print("Predicted Classes:", y_pred_class.tolist())
print("Predicted Probabilities:", y_pred.tolist())

Test Accuracy: 0.7953890489913544
Predicted Classes: [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0

In [33]:
# saving the models weight

model_path = "model_weights.pth"
torch.save(model.state_dict(), model_path)

In [36]:
!ls /content/


drive  model_weights.pth  sample_data


In [35]:
from google.colab import files
files.download("model_weights.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>